Ejercicio: Mejora de un modelo de regresión logística

En el ejercicio anterior, ajustamos un modelo de regresión logística simple para predecir la posibilidad de una avalancha. Esta vez, mejoraremos su rendimiento mediante el uso inteligente de múltiples funciones.

Visualización de datos
Carguemos nuestros datos.

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/avalanche.csv
import graphing # custom graphing code. See our GitHub repo for details

#Import the data from the .csv file
dataset = pandas.read_csv('avalanche.csv', delimiter="\t", index_col=0)

# Split our data into training and test
import sklearn.model_selection
train, test = sklearn.model_selection.train_test_split(dataset, test_size=0.25, random_state=10)

print("Train size:", train.shape[0])
print("Test size:", test.shape[0])

#Let's have a look at the data
print(train.head())

Tenemos numerosas características disponibles:

surface_hoar es cuán perturbada está la superficie de la nieve
fresh_thickness es el grosor de la capa superior de nieve, o 0 si no hay nieve fresca encima
viento es la velocidad máxima del viento ese día, en km/h
débil_capas es el número de capas de nieve que no están bien unidas a otras capas
no_visitors es el número de excursionistas que estuvieron en el sendero ese día
tracked_out es un 1 o un 0. Un 1 significa que la nieve ha sido pisoteada fuertemente por los excursionistas
Regresión logística simple
Hagamos un modelo de regresión logística simple y evalúemos su desempeño con precisión.

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as smf

# Perform logistic regression.
model = smf.logit("avalanche ~ weak_layers", train).fit()

# Calculate accuracy
def calculate_accuracy(model):
    '''
    Calculates accuracy
    '''
    # Make estimations and convert to categories
    avalanche_predicted = model.predict(test) > 0.5

    # Calculate what proportion were predicted correctly
    # We can use sklearn to calculate accuracy for us
    print("Accuracy:", accuracy_score(test.avalanche, avalanche_predicted))

calculate_accuracy(model)

Veamos cómo podemos mejorar nuestro modelo.

Utilizando múltiples características
La mayoría de nuestras funciones parecen ser útiles, al menos en teoría. Probemos un modelo con todas las características que tenemos disponibles.

In [ ]:
# Perform logistic regression.
model_all_features = smf.logit("avalanche ~ weak_layers + surface_hoar + fresh_thickness + wind + no_visitors + tracked_out", train).fit()
calculate_accuracy(model_all_features)

Esa es una gran mejora con respecto al modelo más simple con el que hemos estado trabajando.

Para entender por qué, podemos mirar la información resumida

In [ ]:
model_all_features.summary()

Eche un vistazo a la columna P y recuerde que los valores inferiores a 0,05 significan que podemos estar seguros de que este parámetro está ayudando al modelo a realizar mejores predicciones.

Tanto surface_hoar como wind tienen valores muy pequeños aquí, lo que significa que son predictores útiles y probablemente explican por qué nuestro modelo funciona mejor. Si miramos la columna coef (que indica los parámetros) vemos que estos tienen valores positivos. Esto significa que los vientos más fuertes y una mayor cantidad de capa superficial dan como resultado un mayor riesgo de avalanchas.

Simplificando nuestro modelo
Mirando el resumen nuevamente, podemos ver que tracked_out (cuán pisoteada está la nieve) y fresh_thickness tienen valores p grandes. Esto significa que no son predictores útiles. Veamos qué pasa si los eliminamos de nuestro modelo:

In [ ]:
# Perform logistic regression.
model_simplified = smf.logit("avalanche ~ weak_layers + surface_hoar + wind + no_visitors", train).fit()
calculate_accuracy(model_simplified)

¡Nuestro nuevo modelo funciona de manera muy similar al anterior! En algunas circunstancias, simplificar un modelo como este puede incluso mejorarlo, ya que es menos probable que se sobreajuste.

Selección cuidadosa de características
Por lo general, no elegimos características a ciegas. Pensemos en lo que acabamos de hacer: eliminamos la cantidad de nieve fresca que había en un modelo, tratando de predecir avalanchas. Algo parece estar mal. ¿Seguramente las avalanchas son mucho más probables después de que haya nevado? De manera similar, la cantidad de personas en la pista parece no estar relacionada con la cantidad de avalanchas que hubo, pero sabemos que las personas a menudo pueden desencadenar avalanchas.

Repasemos de nuevo nuestro modelo anterior:

In [ ]:
model_all_features.summary()

Mire la fila fresh_thickness. Se nos dice que tiene un coeficiente negativo. Esto significa que a medida que aumenta el espesor, disminuyen las avalanchas.

De manera similar, no_visitors tiene un coeficiente negativo, lo que significa que menos excursionistas significan más avalanchas.

¿Cómo puede ser esto? Bueno, mientras que los visitantes pueden causar avalanchas si hay mucha nieve fresca, presumiblemente no pueden hacerlo tan fácilmente si no hay nieve fresca. Esto significa que nuestras características no son completamente independientes.

Podemos decirle al modelo que intente tener en cuenta que estas características interactúan, usando un signo de multiplicación. Probemos eso ahora.

In [ ]:
# Create a model with an interaction. Notice the end of the string where
# we've a multiply sign between no_visitors and fresh_thickness
formula = "avalanche ~ weak_layers + surface_hoar + wind + no_visitors * fresh_thickness"
model_with_interaction = smf.logit(formula, train).fit()
calculate_accuracy(model_with_interaction)

¡El modelo ha mejorado hasta un 84 % de precisión! Veamos la información resumida:

In [ ]:
model_with_interaction.summary()

Podemos ver que el término de interacción es útil: el valor p es inferior a 0,05. El modelo también está funcionando mejor que nuestros intentos anteriores.

Hacer predicciones con múltiples características
Muy rápidamente, exploremos lo que significa esta interacción observando las predicciones del modelo.

Primero graficaremos dos características independientes en 3D. Comencemos con las capas débiles y el viento:

In [ ]:
graphing.model_to_surface_plot(model_with_interaction, ["weak_layers", "wind"], test)

El gráfico es interactivo: gírelo y explore cómo existe una relación clara en forma de S entre las características y la probabilidad.

Veamos ahora las características que hemos dicho que pueden interactuar:

In [ ]:
graphing.model_to_surface_plot(model_with_interaction, ["no_visitors", "fresh_thickness"], test)

¡Se ve bastante diferente al otro! Desde cualquier lado, podemos ver una forma de S, pero se combinan de formas extrañas.

Podemos ver que el riesgo aumenta en los días con muchos visitantes y mucha nieve. No existe un riesgo real de avalancha cuando hay mucha nieve pero no hay visitantes, o cuando hay muchos visitantes pero no hay nieve.

El hecho de que muestre un alto riesgo cuando no hay nieve fresca y no hay visitantes podría deberse a la lluvia, que mantiene alejados a los visitantes y las nubes de nieve, pero da como resultado avalanchas de la nieve más vieja. Para confirmar esto, necesitaríamos explorar los datos con mayor profundidad, pero nos detendremos aquí por ahora.

Resumen<br>
¡Bien hecho! Recapitulemos. hemos:

mejorado nuestro modelo simple agregando más funciones.<br>
practicó la interpretación de los coeficientes (parámetros) de nuestro modelo a partir del resumen del modelo<br>
eliminado características innecesarias<br>
exploró cómo a veces es importante pensar en lo que realmente significan sus datos<br>
creó un modelo que combinó características para dar un resultado superior